In [1]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Cake
# 1. The cake is in the kitchen
# 2. The cake is delicious
# 3. The cake is not eaten
# 4. The cake is sweet
# 5. The cake is not sweet
# 6. The cake is not delicious
# 7. The cake is eaten
# 8. The cake is in the stomach

def eat():
    return "eat_cake"

def bake():
    return "bake_cake"

stmap = { 
        Strips(eat(), { "cake": True}, {"cake": False}),
        Strips(bake(), { "cake": False}, {"cake": True})
    }

# all possible actions
feature_domain_dict = { "cake": { True, False} }
strips_domain = STRIPS_domain(feature_domain_dict, stmap)
cake_problem = Planning_problem(strips_domain, { "cake": False}, { "cake": True})
SearcherMPP(Forward_STRIPS(cake_problem)).search()


Solution: {'cake': False}
   --bake_cake--> {'cake': True} (cost: 1)
 2 paths have been expanded and 0 paths remain in the frontier


{'cake': False}
   --bake_cake--> {'cake': True}

In [9]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

def move(x, y, nx, ny):
    return 'move_from_'+str(x)+str(y)+'_to_'+str(nx)+str(ny)

def attack(x, y):
    return "attack_dragon_at_" + str(x) + "_" + str(y)

def open():
    pass

def collectFire(x, y):
    return "collect_fire_at_" + str(x) + "_" + str(y)

def collectEarth(x, y):
    return "collect_earth_at_" + str(x) + "_" + str(y)

def buildFireball():
    return "build_fireball"

# nxn grid with player, castle, walls, and dragons
# End goal is kill the dragon and react the castle
# To kill the dragon, player has to be at dragon coordinates and has the fireball
# To build a fireball, the player needs k earth and l fire
# To attack, the player has to be at the same position as a dragon
# Player can open a chest but idk why - skip for now

def initWorld(n, dragon_coords, fire_coords, earth_coords):
    # nxn grid
    feature_domain_dict = { 
        "player": [(i, j) for i in range(n) for j in range(n)], 
        "dragon": (True, False),
        "fire": (True, False),
        "earth": (True, False),
        "fireball": (True, False), 
    }

    # all possible player movements
    stmap = {
        Strips(move(x, y, x + dx, y + dy), { "player": (x, y)}, { "player": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(n) 
            for y in range(n)
            if x + dx in range(n) and y + dy in range(n)
    }

    # collecting fire
    stmap.update({
        Strips(collectFire(fire_coords[0], fire_coords[1]), { "player": fire_coords, "fire": False}, { "fire": True })
    })

    # collecting earth
    stmap.update({
        Strips(collectEarth(earth_coords[0], earth_coords[1]), { "player": earth_coords, "earth": False}, { "earth": True })
    })

    # building a fireball
    stmap.update({
        Strips(buildFireball(), { "fire": True, "earth": True, "fireball": False}, { "fireball": True, "fire": False, "earth": False})
    })

    # attacking a dragon
    stmap.update({
       Strips(attack(dragon_coords[0], dragon_coords[1]), { "player": dragon_coords, "dragon": True, "fireball": True}, { "dragon": False, "fireball": False }) 
    })

    return STRIPS_domain(feature_domain_dict, stmap)

dragon = (1, 3)
fire = (2, 2)
earth = (3, 3)
world1 = initWorld(5, dragon, fire, earth)
player_coords = (1, 1)
castle = (4, 4)

problem = Planning_problem(
    world1, 
    { "player": player_coords, "dragon": True, "fire": False, "earth": False, "fireball": False}, 
    { "player": castle, "dragon": False}
)

sol = SearcherMPP(Forward_STRIPS(problem)).search()
print(sol)

Solution: {'player': (1, 1), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --move_from_11_to_21--> {'player': (2, 1), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --move_from_21_to_22--> {'player': (2, 2), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --collect_fire_at_2_2--> {'player': (2, 2), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --move_from_32_to_33--> {'player': (3, 3), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --collect_earth_at_3_3--> {'player': (3, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --move_from_33_to_23--> {'player': (2, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --move_from_23_to_13--> {'player': (1, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --build_fire

In [5]:
# ; Shakey's world, or versions thereof, is an AI classic. A robot, named Shakey, is moving around in a set of rooms, 
# ; connected by doors. 
# ; In the rooms there are light switches, so that the light in the room can be on or off. Spread throughout this world 
# ; there are a number of big boxes and a number of smaller balls. 
# ; Also there are specific locations in each room where the boxes and robots can be present initially(init1,init2,init3) and where they
# ; need to be transfered finally(fin1,fin2,fin3).
# ; Here's an example of a rather small world layout: 
# ;
# ; The follwing restrictions apply to Shakey's actions: 
# ;
# ; Shakey can carry small balls, but only one at a time because he has only one gripper. 
# ; For Shakey to be able to find a door and  move to another room, the light must be on in that room. 
# ; Shakey can not carry boxes, but can push them around. 
# ; To switch the light in a room on or off Shakey must putdown the ball if it was carrying otherwise he can not 
# ; turn on the light switch. 

from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

def pickup(x, y):
    return f"pickup_ball_at_{x}_{y}"

def putdown(x, y):
    return f"putdown_at_{x}_{y}"

def pushbox(x1, y1, x2, y2):
    return f"pushbox_from_{x1}_{y1}_to_{x2}_{y2}" 

def move_bet_rooms(r1, r2):
    return f"tp_from_{r1}_to_{r2}"

def move_in_room(x1, y1, x2, y2):
    return f"move_from_{x1}_{y1}_to_{x2}_{y2}"

def turn_on_light(r1):
    return f"turn_on_in_room_{r1}"


def initProblemDefinition(size, init_position, light_coords):
    rooms = ["room1", "room2"]

    if (init_position[0] not in range(size) or
        init_position[1] not in range(size)):
        raise ValueError("Invalid initial position")

    if (light_coords[0] not in range(size) or
        light_coords[1] not in range(size)):
        raise ValueError("Invalid light coordinates")

    # bidirectional connection between rooms
    adj = {
        "room1": ["room2"],
        "room2": ["room1"]
    }

    feature_domain_dict = {
        "room": ["room1", "room2"],
        "robot": [(x, y) for x in range(size) for y in range(size)],
    }

    for room in rooms:
        feature_domain_dict[room] = False

    # defining robot movements within the room
    stmap = {
        Strips(move_in_room(x, y, x + dx, y + dy), { "robot": (x, y)}, { "robot": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(size) 
            for y in range(size)
            if x + dx in range(size) and y + dy in range(size)
    }

    # turning on light
    # the robot has to be at lights coords to turn them on
    stmap.update({
        Strips(turn_on_light(room), { "room": room, room: False, "robot": light_coords }, { room: True })
        for room in rooms
    })

    # move between rooms definied in adj list
    # it is allowed only if the light is turned on
    # starting position in the new room is `init_position`
    stmap.update({  
        Strips(move_bet_rooms(fr, to), { "room": fr, fr: True }, { "room": to, "robot": init_position }) 
        for fr, adj_list in adj.items() for to in adj_list
    })

    return STRIPS_domain(feature_domain_dict, stmap)


size = 5
light_coords = (0, 0)
init_position = (2, 2)

problem_definition = initProblemDefinition(size, light_coords, init_position)

problem = Planning_problem(
    problem_definition, 
    { "robot": init_position, "room1": False, "room2": False, "room": "room1" }, 
    { "room1": True, "room2": True }
)

sol = SearcherMPP(Forward_STRIPS(problem)).search()
print(sol)

Solution: {'robot': (2, 2), 'room1': False, 'room2': False, 'room': 'room1'}
   --turn_on_in_room_room1--> {'robot': (2, 2), 'room1': True, 'room2': False, 'room': 'room1'}
   --tp_from_room1_to_room2--> {'robot': (0, 0), 'room1': True, 'room2': False, 'room': 'room2'}
   --move_from_0_0_to_1_0--> {'robot': (1, 0), 'room1': True, 'room2': False, 'room': 'room2'}
   --move_from_1_0_to_1_1--> {'robot': (1, 1), 'room1': True, 'room2': False, 'room': 'room2'}
   --move_from_1_1_to_2_1--> {'robot': (2, 1), 'room1': True, 'room2': False, 'room': 'room2'}
   --move_from_2_1_to_2_2--> {'robot': (2, 2), 'room1': True, 'room2': False, 'room': 'room2'}
   --turn_on_in_room_room2--> {'robot': (2, 2), 'room1': True, 'room2': True, 'room': 'room2'} (cost: 7)
 69 paths have been expanded and 20 paths remain in the frontier
{'robot': (2, 2), 'room1': False, 'room2': False, 'room': 'room1'}
   --turn_on_in_room_room1--> {'robot': (2, 2), 'room1': True, 'room2': False, 'room': 'room1'}
   --tp_from_room